<a href="https://colab.research.google.com/github/yannickfunk/EmbeddedMLLab/blob/main/end2end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys


# if in colab, make colab setup
if 'google.colab' in sys.modules:
    FILENAME = "end2end.ipynb"
    BRANCH = "main"
    !git clone -b $BRANCH https://github.com/yannickfunk/EmbeddedMLLab tmp
    !rm tmp/$FILENAME
    !mv tmp/* .
    !rm -rf tmp
    !rm -rf sample_data
    !pip install -r requirements.txt

In [2]:
import torch
import tqdm

from utils.dataloader import VOCDataLoader
from models.tinyyolov2 import TinyYoloV2
from utils.yolo import nms, filter_boxes
from utils.viz import display_result

loader = VOCDataLoader(train=False, batch_size=1)

# make an instance with 20 classes as output
net = TinyYoloV2(num_classes=20)

# load pretrained weights
sd = torch.load("voc_pretrained.pt")
net.load_state_dict(sd)

#put network in evaluation mode
net.eval()
for idx, (input, target) in tqdm.tqdm(enumerate(loader), total=len(loader)):

    #input is a 1 x 3 x 320 x 320 image
    output = net(input)
    "output is of a tensor of size 32 x 125 x 10 x 10"
    #output is a 32 x 125 x 10 x 10 tensor

    #filter boxes based on confidence score (class_score*confidence)
    output = filter_boxes(output, 0.1)

    #filter boxes based on overlap
    output = nms(output, 0.25)

    display_result(input, output, target, file_path='yolo_prediction.png')

  0%|          | 0/1999639040 [00:00<?, ?it/s]

Extracting data/VOCtrainval_11-May-2012.tar to data/


FileNotFoundError: [Errno 2] No such file or directory: 'voc_pretrained.pt'